In [82]:
from os import listdir
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, Sequential, losses

In [3]:
def data_preprocess(path):
    files=listdir(path)
    data=[]
    stemmer=PorterStemmer()
    translator = str.maketrans('','' , string.punctuation)
    stop_words = set(stopwords.words('english'))
    for file in files:
        review_updated=[]
        file1=open(path+file,encoding="utf8")
        review=file1.read()
        review=review.lower()
        words = nltk.word_tokenize(review)
        for w in words:
            x = w.translate(translator)
            if (x not in stop_words and x != '') :
                stem_word=stemmer.stem(x)
                review_updated.append(stem_word)
        data.append(review_updated)
    return data

In [81]:
file1=open("./aclImdb/imdb.vocab",encoding="utf8")
vocab=file1.read()
vocab = vocab.split("\n")

In [4]:
train_pos_data=data_preprocess('./aclImdb/train/pos/')
train_neg_data=data_preprocess('./aclImdb/train/neg/')

In [12]:
print(((train_pos_data[0])))

['bromwel', 'high', 'cartoon', 'comedi', 'ran', 'time', 'program', 'school', 'life', 'teacher', '35', 'year', 'teach', 'profess', 'lead', 'believ', 'bromwel', 'high', 'satir', 'much', 'closer', 'realiti', 'teacher', 'scrambl', 'surviv', 'financi', 'insight', 'student', 'see', 'right', 'pathet', 'teacher', 'pomp', 'petti', 'whole', 'situat', 'remind', 'school', 'knew', 'student', 'saw', 'episod', 'student', 'repeatedli', 'tri', 'burn', 'school', 'immedi', 'recal', 'high', 'classic', 'line', 'inspector', 'sack', 'one', 'teacher', 'student', 'welcom', 'bromwel', 'high', 'expect', 'mani', 'adult', 'age', 'think', 'bromwel', 'high', 'far', 'fetch', 'piti', 'nt']


In [10]:
train_pos_df = pd.DataFrame(train_pos_data)

In [39]:
train_pos_df = pd.DataFrame({'Review': [row for row in train_pos_data]})
train_pos_df['Label']=1

train_neg_df = pd.DataFrame({'Review': [row for row in train_neg_data]})
train_neg_df['Label']=0

train_data = pd.concat([train_pos_df, train_neg_df], ignore_index=True)

In [59]:
x_data=train_data.iloc[:,0]
y_data=train_data.iloc[:,1]

In [60]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.20, random_state=42)

In [85]:
ev_features = X_train.shape[0]
vocab_size = len(vocab)

In [92]:
# BUilding the LSTM Network

lstm_nlp_model = Sequential()
lstm_nlp_model.add(layers.Embedding(vocab_size, ev_features, input_length = X_train.shape[0]))
lstm_nlp_model.add(layers.LSTM(128, activation='relu',return_sequences=True))
lstm_nlp_model.add(layers.Dropout(0.2))
lstm_nlp_model.add(layers.LSTM(128,activation='relu'))
lstm_nlp_model.add(layers.Dropout(0.2))
lstm_nlp_model.add(layers.Dense(32,activation='relu'))
lstm_nlp_model.add(layers.Dropout(0.2))
lstm_nlp_model.add(layers.Dense(1)) #add softmax maybe

lstm_nlp_model.compile(loss='BinaryCrossentropy',optimizer='adam',metrics=['accuracy'])